# Monte Carlo Simulation for Estimating Pi

Monte Carlo methods are widely used for numerical simulations, optimization problems, and probabilistic modeling.
One of the classic problems that showcase the power of Monte Carlo simulations is estimating the value of π.

The idea is simple: randomly generate points inside a square and count how many fall within a quarter-circle.
The ratio of points inside the quarter-circle to the total number of points gives an approximation of π.

## Why Parallelization Helps

Each experiment (random point generation and checking whether it falls inside the circle) is completely independent of the others. This makes it an ideal use case for parallelization since multiple workers can compute the results simultaneously without needing to communicate.

By using `RayManager`, we can distribute the computation across multiple CPU cores, significantly improving performance compared to a sequential implementation.

## Define the Computational Task

Each task generates a set of random points and counts how many land inside the quarter-circle. This logic is implemented in a `MonteCarloPiTask` class that extends `RayTask`.

In [ ]:
import random
from raygent import RayTask

class MonteCarloPiTask(RayTask):

    def process_item(self, item: int) -> int:
        """Simulates a Monte Carlo experiment for estimating Pi.

        Generates `num_samples` random points and counts how many fall inside the quarter-circle.
        """
        inside_circle = 0
        for _ in range(item):
            x, y = random.uniform(0, 1), random.uniform(0, 1)
            if x**2 + y**2 <= 1:
                inside_circle += 1
        return inside_circle

Here, the method `process_item` takes an integer (a dummy input, as it’s only needed to trigger the computation), generates `num_samples` random points, and returns the count of points that landed inside the quarter-circle.

## Running the Task with `RayManager`

To estimate π, we run multiple Monte Carlo simulations in parallel. Each task will generate `num_samples` points, and we will sum up the results from all workers to get the final estimate.


In [2]:
from raygent import RayManager


def estimate_pi(n_workers: int, samples_per_worker: int) -> float:
    """Estimates Pi using Monte Carlo simulation with parallel execution.

    Args:
        n_workers: Number of independent Monte Carlo simulations to run.
        samples_per_worker: Number of points each worker will generate.

    Returns:
        float: Estimated value of π.
    """
    manager = RayManager(MonteCarloPiTask, n_cores=n_workers, use_ray=True)

    # We use `0` since tasks don't require input data
    manager.submit_tasks(items=[samples_per_worker] * n_workers, at_once=False)
    results = manager.get_results()

    total_inside_circle = sum(results)
    total_samples = samples_per_worker * n_workers

    pi_estimate = (total_inside_circle / total_samples) * 4
    return pi_estimate


In [3]:
# Example Usage
estimated_pi = estimate_pi(n_workers=8, samples_per_worker=1_000_000)
print(f"Estimated π: {estimated_pi}")

2025-02-27 16:41:13,264	INFO worker.py:1841 -- Started a local Ray instance.


Estimated π: 3.141148
